In [10]:
import json
import pandas as pd 
import numpy as np 
from glob import glob
import os
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from PIL import Image


In [3]:
folder_path=glob("../../data/*")
data1_img_path=glob(folder_path[0]+"/data/*.jpg")
data2_img_path=glob(folder_path[2]+"/data/*.jpg")


def AnnotationsPd(annotations):
    values_list = []
    names_list = [] 
    df = pd.DataFrame(annotations['tags'])
    for row in df['attributes']:
        values_list.append(row[0]['value'])
        names_list.append(row[0]['name'])

    df=df.drop(['attributes'],axis=1)   
    df.insert(3,'value',values_list)
    df.insert(4,'name',names_list)
    return df


In [5]:
data1_img_path.extend(data2_img_path)


In [7]:
Train_file_path=[]

with open(folder_path[0]+'/annotations.json') as f:
    annotations_json = json.load(f)
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in data1_img_path if str(frame).zfill(5) in s] 
    Train_file_path.append(file[0])
with open(folder_path[2]+'/annotations.json') as f:
    annotations_json = json.load(f)   
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in data2_img_path if str(frame).zfill(5) in s] 
    Train_file_path.append(file[0])


Train_file_path=list(set(Train_file_path))
Train_TF_standard=[]
for i in range(len(Train_file_path)):
    shutil.copy(Train_file_path[i],'../../data/6_Train_styleGAN/img/Abnormal/'+str(i)+'.jpg')

In [8]:
Train_not_standard_file1=list(set(data1_img_path)-set(Train_file_path))
for i in range(len(Train_not_standard_file1)):
    shutil.copy(Train_not_standard_file1[i],'../../data/6_Train_styleGAN/img/Normal/'+str(i)+'.jpg')


In [11]:
standard_data=glob('../../data/6_Train_styleGAN/img/Abnormal/*.jpg')
not_standard_data=glob('../../data/6_Train_styleGAN/img/Normal/*.jpg')

def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

def padding(standard_data):

    for i in range(len(standard_data)):
        image=Image.open(standard_data[i])
        image=image.crop((196,128,508,640))
        image=image.resize((312,512))
        image=expand2square(image,(0,0,0))
        image.save(standard_data[i])

In [12]:
padding(standard_data)
padding(not_standard_data)